<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>

<font color='blue'>
<br />
Никита, доброго дня! рад приветствовать тебя) <br />
(а дальше шаболннная обящательная часть, как ты заметил по другим спринтам)
<br />    

<br />
Меня зовут Николай Шавлюго. <br />И на этом этапе твоего движения к новой профессии от меня будут комментарии по написанному коду. <br />Чтобы меньше "мусорить эфир" и чтобы максимально наглядно отделяться от основного кода, есть предложение условиться в некоторых обозначениях:<br /> <br />
<u><b>ТАКОЙ ШРИФТ</b></u> - всегда начало комментария <br />
<font color='green'>такой шрифт</font> - комментарии о том, что всё ОК <br/>
<font color='orange'>такой шрифт</font> - комментарии о том, что всё ОК по результату,<br> однако есть на что обратить внимание в плане применения техник, или есть способы сделать более короткий или быстрый код <br/>
<font color='red'>такой шрифт</font> - комментарии о том, что есть критичный момент, влияющий на бизнес-результат проекта.<br/>
</font><br /><br />
<font color='green'>Моей целью является не "уличить" в не знании, а просто высказать сверху твоих знаний - свой опыт, что бы ты мог использовать его для своего дальнейшего успеха) И очень здорово будет, если тебе удастся задавать вопросы, да и вообще - всячески доставать меня, если я по каким-то причинам не приму проект:) При этом, из своего опыта скажу, хорошо и важно, когда переписка ревьюера и студента - сохраняется на следующие проекты и даже на будущую практическую деятельность.<br><BR> В ПУТЬ!<br>

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<span style="background-color: #FFFFCC">Никак не получается добиться результата f1_score на тесте. Подскажите, что делаю не так. На CatBoost на тренировочной выборке обучаетсяя до f1 в районе 0.99, а на тесте низкие результаты</span>

<span style="background-color: #00FF00">Всё получилось, спасибо!</span>

<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='blue'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА 2</u></b>
</font>
<font color='green'><br>Привет, Артём!<br>
ОК, здорово.<br>
Спасибо за проект) УСПЕШНОГО ДАЛЬНЕЙШЕГО ОБУЧЕНИЯ!

__Загрузка модулей__

In [40]:
import numpy as np
import pandas as pd

# обработка текстов
import re 
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords') 
from nltk.stem import WordNetLemmatizer

#обработка данных
from tqdm import tqdm
from tqdm import notebook
from joblib import Parallel, delayed

# игнор предупреждений
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# обучение модели 
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Оглавление
---

##### Часть 1. Подготовка:
* [1.1. Загрузка данных.](#1-bullet)
* [1.2. Лемматизация и токенизация.](#2-bullet)


##### Часть 2. Обучение:
* [2.1. Векторизация.](#3-bullet)
* [2.2. Построение модели.](#4-bullet)

##### Часть 3. Вывод.


<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='blue'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>
</font>
<font color='green'><br>
план понятен)

# 1. Подготовка

__1.1 Загрузка данных__
<a id='1-bullet'></a>

Удаляем дубликаты и пропущенные значения.

In [2]:
df=pd.read_csv('/datasets/toxic_comments.csv')
df = df.dropna().drop_duplicates()

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Создаем список признаков и целевого значения.

In [4]:
texts=list(df['text'])
target =df.toxic.values

<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='blue'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>
</font>
<font color='green'><br>
ок, данные на базе)

__1.2. Лемматизация и токенизация__
   <a id='2-bullet'></a> 
- оставляем в тексте только латинские символы
- лемматизируем с помощью pymystem3
- удаляем стоп-слова
- разделяем на токены

1. WordNetLemmatizer

In [6]:
m = WordNetLemmatizer()
from nltk.tokenize import word_tokenize
def lemmatize(text):
    doc=[]
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.strip(' ')
    lem_text = [m.lemmatize(i) for i in word_tokenize(text)]
    lem_text = " ".join(lem_text)
    return lem_text
  
processed_texts = Parallel(n_jobs=-1)(delayed(lemmatize)(i) for i in tqdm(texts))

100%|██████████| 159571/159571 [03:22<00:00, 786.49it/s] 


__Вывод__ WordNetLemmatizer работает в разы быстрее остальных.

Создаем новую таблицу из полученных значений

In [7]:
new_df = pd.Series(processed_texts)
new_df = pd.concat([new_df,df.toxic], axis=1)
new_df.columns = ['text','toxic']

In [8]:
print(new_df.head())

                                                text  toxic
0  Explanation Why the edits made under my userna...      0
1  D aww He match this background colour I m seem...      0
2  Hey man I m really not trying to edit war It s...      0
3  More I can not make any real suggestion on imp...      0
4  You sir are my hero Any chance you remember wh...      0


# 2. Обучение

__2.1. Векторизация__
<a id='3-bullet'></a> 

In [31]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
count_tf_idf.fit(X_TRAIN)
f_train = count_tf_idf.transform(X_TRAIN)
f_test = count_tf_idf.transform(X_TEST)

<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='blue'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>
</font>
<font color='green'><br>


__2.2. Построение модели__
<a id='4-bullet'></a> 

__Разделение на тестовую и тренировочную выборки__

In [20]:
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(new_df['text'].values,new_df.toxic.values, test_size = 0.2,stratify=new_df.toxic.values)

__Выравнивание классов__
Количество токсичных твитов значительно меньше чем, не токсичных, что может вызвать переобучение модели

In [21]:
cw = [1,(Y_TRAIN==0).sum() / (Y_TRAIN==1).sum()]
print(cw)
a={}
a[1] = cw[0]
a[0] = cw[1]
print(a)

__GridSearch для CatBoost__
Подберем лучшие параметры для модели CatBoostClassifier

In [22]:
model = CatBoostClassifier(loss_function='Logloss',early_stopping_rounds=3)

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_search_result = model.grid_search(grid, 
                                       X=x_train, 
                                       y=y_train,
                                       stratified=True)

__Вывод__
Лучший результат: depth=4, l2_leaf_reg = 9,learning_rate = 0.1

__CatBoostClassifier__

In [37]:
model = CatBoostClassifier(depth=4, l2_leaf_reg = 9,class_weights=cw,learning_rate = 0.1, eval_metric='F1',verbose=200)
model.fit(f_train, Y_TRAIN)
print(f1_score(model.predict(f_test),Y_TEST))

0:	learn: 0.4311382	total: 3.42s	remaining: 57m 1s
200:	learn: 0.8629054	total: 9m 28s	remaining: 37m 41s
400:	learn: 0.8921253	total: 18m 46s	remaining: 28m 2s
600:	learn: 0.9083995	total: 27m 57s	remaining: 18m 33s
800:	learn: 0.9190818	total: 37m 14s	remaining: 9m 15s
999:	learn: 0.9274896	total: 46m 32s	remaining: 0us
0.751145038167939


__Вывод__ Лучший результат на тестовой выборке :0.751

LGBMClassifier

__Логистическая регрессия__

In [39]:
logit = LogisticRegression(random_state=12345, penalty='l1', solver='saga', max_iter=1000,multi_class='ovr')

logit.fit(f_train, Y_TRAIN)
preds_lr = logit.predict(f_test)
print(f1_score(preds_lr,Y_TEST))

0.7734265734265734


__Вывод__ Лучший результат на тестовой выборке :0.773

# 3. Вывод

- WordNetLemmatizer дает наиболее быструю обработку текстов.
- CatBoostClassifier (с подбором параметром GridSearch) и логистическая регрессия выполняют заданное требование, однако при обучении модели логистической регрессии результат f1_score на тестовой выборке чуть выше.
- TF-IDF vectorizer даёт достаточно быструю обработку данных и хорошие результаты при обучении модели.


<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='blue'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>
</font>
<font color='green'><br>
Никита, 
Идейные шаги  у тебя - ОК и это очень хорошо!!!. Всё дело в деталях (как обычно).<br>
Ниже я приведу алгоритм, по которому предлагаю тебе скорректировтаь код. Т.е. даже сократить код - лучше сказать)<br>
В нём только основные шаги, с относительно простыми реализациями. Попробуй так, это долно помочь достчь результат. И заодно увидеть где есть перекос в превичном коде. (мне его не прогнать сейчас - всё время умирает расчёт..)<br>
Идейно так: не всё сложные преобразования и модели могут помочь, лучше начинать с простого, смотреть что не работает, а потом наращивать слоность...<br> 

<div style="background: #cceeaa; padding: 5px; border: 1px solid green; border-radius: 5px;">
    <font color='blue'> <b><u>КОММЕНТАРИЙ РЕВЬЮЕРА</u></b>
</font>
<font color='green'><br>
1) Загрузить данные - это ОК у тебя.<br>
2) Провести чистку регулярными выражениями.<br>
Например такими (тут всякое можно ещё посмотерть):<br>
def clean_text(text):<br>
    text = re.sub(r"can't", "cannot ", text)<br>
    text = re.sub(r"n't", " not ", text)<br>
    text = re.sub(r"\'re", " are ", text)<br>
    text = re.sub(r"\'d", " would ", text)<br>
    text = re.sub(r'[^a-zA-Z]', ' ', text)<br>
    text = text.strip(' ')<br>
    return text<br><br>

3) Провести лемматизацию (НО КСТАТИ - ЭТО МОЖНО И НЕ ДЕЛАТЬ - ИТОЖЕ ДОЛЖЕН БЫТЬ РЕЗУЛЬТАТ)<br>
def w_lemmatize(text):<br>
    lem_text = [lemmatizer.lemmatize(i) for i in w_tokenizer.tokenize(text)]<br>
    lem_text = " ".join(lem_text)<br>
    return lem_text    <br><br>
    
4) провести векторизацию.<BR>
nltk.download('stopwords')<BR>
stopwords = set(nltk_stopwords.words('english'))<BR>
count_tf_idf = TfidfVectorizer(stop_words=stopwords)<BR>
count_tf_idf.fit(corpus_train)<BR>
f_train = count_tf_idf.transform(corpus_train)<BR>
f_test = count_tf_idf.transform(corpus_test)<BR><BR>
    
5) А дальше логистическая регерессия, например<br>
можно попробовать с праметрами "из коробки" - должно сработать даже так.<br>
Если нет - попробовать гридсечем прогнать, например nfrbt gfhfvtnhs:<br>
param_lr = {'random_state': [100500], 'solver': ['liblinear'], 'penalty': ['l1', 'l2'], \<br>
            'class_weight': ['balanced'], 'C': [5, 10]}<br><br>
    
В итоге метрика должна быть в районе 0,77.<br>

БУДУ ЖДАТЬ ТВОИ КОММЕНТАРИЕВ!

# Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны